This notebook will be a tutorial in how to calculate volume, as a difference between consecutive DEMs.
A rough overview of the steps to be followed is given below-
1. Sort DEMs in chronological order. 
2. Inspect DEMs and Mosaics to check for distortions. 
3. Create Vector Mask to remove Areas with distortions. 
4. Mask original DEM with created vector mask. 
5. Use grass's `r.fillnulls` command to create the gaps left by the masked distortions. 
6. Obtain set of error free DEMs.
7. Using this set, obtain delta DEMs between consecutive DEMs, or as per requirement. 
8. Do summation over obtained Delta DEM and multiply by area of a pixel, as per scale to obtain volume. 


*Note- Steps with a * \* *indicates that there are algorithms/variables to be tuned*

In [2]:
#Added for vector processing
import ogr
import os
import json 
import osr
import vector_processing.point_processing as pp
#Old imports for raster processing 
import gdal
import cv2
from subprocess import call
import numpy as np
import matplotlib.pyplot as plt

#Custom Written Scripts
try:
    import dem_proj as dp
    import dem_filtering as df
    import dem_perf_check as dpc
    from raster_chunks import GeoChunks as gc 
    
except ImportError:
    print("Import Error. Check if dem_filtering.py is present in PYTHONPATH")
    print("PYTHONPATH = ")
    call(["echo", "$PYTHONPATH"])

# Bit of formatting to change default inline code style:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

Setting up PATH variables. 

We will continue to set up PATH variables, as and when necessary.



In [2]:
PATH_PREFIX = "/home/madhavm/vimana/workdir/rough/vector_test/"
input_file = PATH_PREFIX + "15may_trim.shp"
output_file = PATH_PREFIX + "b.shp"

Now, we we first need to convert this vector layer to a vector layer with a Projected Coordinate System, (Lat Long will be in UTM)


In [3]:
pp.poly_as_points(polygon_file=input_file,
                 points_file=output_file)

Specified output file already exists.
Deleting current file and replaing it with new points file
	This could lead to possible errors.


In [4]:
p_2

[[array([ 0.89166218, -0.4527014 ]),
  array([ 0.09319471, -0.04731543]),
  [array([  804396.21459791,  1438132.32000357]),
   array([  804396.30779262,  1438132.27268815])]],
 [array([ 0.89167455, -0.45267704]),
  array([ 0.408503, -0.207385]),
  [array([  804396.30779262,  1438132.27268815]),
   array([  804396.71629561,  1438132.06530314])]],
 [array([ 0.88532111, -0.46498014]),
  array([ 67.91225587, -35.66824525]),
  [array([  804396.71629561,  1438132.06530314]),
   array([  804464.62855148,  1438096.3970579 ])]],
 [array([-0.21822709, -0.97589802]),
  array([-10.06377107, -45.0045608 ]),
  [array([  804464.62855148,  1438096.3970579 ]),
   array([  804454.56478041,  1438051.3924971 ])]],
 [array([-0.97282833,  0.23152763]),
  array([-28.55099921,   6.79497605]),
  [array([  804454.56478041,  1438051.3924971 ]),
   array([  804426.0137812 ,  1438058.18747315])]],
 [array([-0.38523363, -0.92281908]),
  array([ -6.18891484, -14.82541555]),
  [array([  804426.0137812 ,  1438058.1874

Note that **unlike** gdal's raster bands, vector layers are 0-indexed.  
Data -> Layers -> Geometry (For Point locations)


In [4]:
driver = ogr.GetDriverByName("ESRI Shapefile")
data_src = driver.Open(PATH_PREFIX + "15may_trimUTM.shp")
layer = data_src.GetLayerByIndex(0)
print(layer.GetFeatureCount())
print(data_src.GetLayerCount())
srs = osr.SpatialReference()
srs.ImportFromEPSG(int(layer.GetSpatialRef().GetAttrValue("AUTHORITY",1)))
print(int(layer.GetSpatialRef().GetAttrValue("AUTHORITY",1)))
out_srs = osr.SpatialReference()


1
1
32643
PROJCS["WGS 84 / UTM zone 43N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",75],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32643"]]


In [21]:
f = layer[0]
dj = json.loads(f.GetGeometryRef().ExportToJson())
print(f.GetGeometryRef().GetGeometryCount())
p_arr = np.array(dj['coordinates'][9])
#print(dj['coordinates'])
print(p_arr.shape)

9


IndexError: list index out of range

Now to test out creating a vector file. 

In [8]:
def define_lines(point_arr):
    lines = []
    for i in range(point_arr.shape[0]-1):
        temp = []
        start_point = point_arr[i]
        end_point = point_arr[(i+1)%point_arr.shape[0]]
        line_vec = end_point - start_point
        unit_vec = line_vec / np.linalg.norm(line_vec)
        temp.append(unit_vec)
        temp.append(line_vec)
        temp.append([start_point,end_point])
        lines.append(temp)
        
    return lines
    

Write function to take np.array of points along polygon as arguments, and return a list of lists. The lists inside the main list, should each define a line segment that defines the polygon. 
Format for the lists should be 
[ [unit_vec],[line_vec], [[start_point], [end_point]] ]


In [9]:
t1 = define_lines(p_arr)

In [10]:
t1

[[array([-0.37369812, -0.92755039]),
  array([-1.85918336, -4.61465059]),
  [array([  804370.30662983,  1438068.75043064]),
   array([  804368.44744648,  1438064.13578006])]],
 [array([ 0.55638455, -0.83092493]),
  array([ 0.33773713, -0.50438892]),
  [array([  804368.44744648,  1438064.13578006]),
   array([  804368.78518361,  1438063.63139114])]],
 [array([ 0.91038858, -0.41375432]),
  array([ 2.91076583, -1.3228878 ]),
  [array([  804368.78518361,  1438063.63139114]),
   array([  804371.69594944,  1438062.30850333])]],
 [array([ 0.99758183,  0.06950176]),
  array([ 0.45121001,  0.03143591]),
  [array([  804371.69594944,  1438062.30850333]),
   array([  804372.14715945,  1438062.33993924])]],
 [array([ 0.79365038,  0.60837413]),
  array([ 0.25219469,  0.1933203 ]),
  [array([  804372.14715945,  1438062.33993924]),
   array([  804372.39935414,  1438062.53325954])]],
 [array([ 0.49563916,  0.86852854]),
  array([ 2.57145097,  4.50605749]),
  [array([  804372.39935414,  1438062.53325954

In [11]:
driver.DeleteDataSource(output_file)
data_src = driver.CreateDataSource(output_file)
samp_dist = 0.1
srs = osr.SpatialReference()
srs.ImportFromEPSG(32643)
l2 = data_src.CreateLayer("hi",srs,ogr.wkbPoint)
field_id = ogr.FieldDefn("id",ogr.OFTInteger)
l2.CreateField(field_id)
for j in range(len(t1)):
    b = t1[j][0] * samp_dist
    start_point = t1[j][2][0]
    a = t1[j][1]
    num_lines = int(np.floor(np.linalg.norm(a) / np.linalg.norm(b)))
    for i in range(num_lines):
        feature = ogr.Feature(l2.GetLayerDefn())
        feature.SetField("id",None)
        t = start_point + b*i
        wkt = "POINT(%f %f)" % (t[0],t[1])
        point = ogr.CreateGeometryFromWkt(wkt)
        feature.SetGeometry(point)
        l2.CreateFeature(feature)
        feature = None
data_src = None


Converting the attributes of a vector file to a CSV file .. 


In [3]:
driver = ogr.GetDriverByName("ESRI Shapefile")
data_src = driver.Open("/home/y13/vimana/test_out.shp")

In [28]:
data_src.GetLayerCount()
layer = data_src.GetLayerByIndex(0)
layer.GetFeatureCount()
f = layer[0]

field_names = []
field_values = []
for i in range(f.GetFieldCount()):
    field_names.append(f.GetFieldDefnRef(i).GetName())
    field_values.append(f.GetField(i))
print(field_names)
print(field_values)


['16june', '20may', '29apr', '03june', '15may', '14feb', '09may']
[883.35956, 883.41852, 883.84326, 883.56702, 883.74951, 883.8573, 883.69879]


In [30]:
import csv

csv_file = open('/home/y13/vimana/out.csv','w')
w = csv.writer(csv_file)
#Writing Header Row to CSV
w.writerow(field_names)
for ftr in layer:
    r = []
    for i in range(ftr.GetFieldCount()):
        r.append(ftr.GetField(i))
    w.writerow(r)
csv_file.close()